# Burst Extractor: Sentinel-1 Burst IDs From Sentinel-1 Scene IDs

Sentinel-1 scenes experience frame shift. Stacks of scenes are not guaranteed to be coregistered, and usually are not.

Sentinel-1 bursts are geostationary, and stacks comprised of bursts are coregistered. 

OPERA data are delivered as bursts. 

This notebook uses the ASF Burst Extractor to identify burst IDs for a given Sentinel-1 scene. 

In [ ]:
import getpass
import requests

import asf_search as disco

In [ ]:
username = input('Username:')
password = getpass.getpass('Password:')

try:
    session = disco.ASFSession().auth_with_creds(username, password)
except disco.ASFAuthenticationError as e:
    print(f'Auth failed: {e}')
else:
    print('Success!')

In [ ]:
scene = input("Enter a Sentinel-1 granule ID")

In [ ]:
beam_modes = ['IW1', 'IW2', 'IW3']
max_bursts_per_swath = range(15)

orbital_path = f"T{disco.granule_search(scene)[1].properties['pathNumber']:03d}"

run = True
while run:
    bursts = []
    run = False
    for beam_mode in beam_modes:
        for i in max_bursts_per_swath:
            url = f'https://sentinel1-burst.asf.alaska.edu/{scene}/{beam_mode}/VV/{i}.xml'
            response = session.get(url)
            status = response.status_code
            if status != 404:
                print(f'{url} {status}')
                if status == 200:
                    print(response.headers['content-disposition'])
                    burst_swath = '-'.join(response.headers['content-disposition'].split("filename=")[-1].split('_')[1:3])
                    bursts.append(f"{orbital_path}-{burst_swath}")
                elif status == 202:
                    run = True

In [ ]:
bursts = [b.replace('-', '_') for b in bursts]
print(f"Found {len(bursts)} bursts") 
bursts